# Cellpy batch 


*Author: Jan Petter Mæhlen, IFE*

*Date: 2018-10-17*

## Setting up the project

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
# import seaborn as sbn # can be used for getting nice colormaps and good settings of the matplotlib rc

In [3]:
from cellpy.utils import batch
from cellpy import prms

### Using batch to load a info_df
This is just to get the filenames and masses

In [4]:
log_level = "INFO" 
b = batch.init(default_log_level=log_level)
json_file = r"C:\Scripting\Processing\Celldata\outdata\SiCANODE\cellpy_batch_sic_2018_minority.json"
b.load_info_df(json_file)

In [5]:
infodf = b.info_df.copy()
del b

## Checking if we can do any lookup in the cellpyfiles

In [6]:
infodf.head()

,cell_type,cellpy_file_names,fixed,groups,labels,loadings,masses,raw_file_names,sub_groups,total_masses
20180925_sic070_01_cc,anode,C:\ExperimentalData\BatteryTestData\Arbin\HDF5...,0,1,sic070_01,2.939529,5.194576,[\\ad.ife.no\dfs\Org\ensys\EnergyStorageMateri...,1,5.82
20180925_sic070_02_cc,anode,C:\ExperimentalData\BatteryTestData\Arbin\HDF5...,0,1,sic070_02,2.681941,4.739381,[\\ad.ife.no\dfs\Org\ensys\EnergyStorageMateri...,2,5.31
20180925_sic070_03_cc,anode,C:\ExperimentalData\BatteryTestData\Arbin\HDF5...,0,1,sic070_03,2.818311,4.980367,[\\ad.ife.no\dfs\Org\ensys\EnergyStorageMateri...,3,5.58
20180925_sic070_04_cc,anode,C:\ExperimentalData\BatteryTestData\Arbin\HDF5...,0,1,sic070_04,3.116305,5.506965,[\\ad.ife.no\dfs\Org\ensys\EnergyStorageMateri...,4,6.17
20180925_sic071_01_cc,anode,C:\ExperimentalData\BatteryTestData\Arbin\HDF5...,0,1,sic071_01,3.130590,5.532209,[\\ad.ife.no\dfs\Org\ensys\EnergyStorageMateri...,5,6.21


In [7]:
import pandas as pd
import matplotlib.pylab as plt

infoname = '/CellpyData/info'
dataname = '/CellpyData/dfdata'
summaryname = '/CellpyData/dfsummary'
fidname = '/CellpyData/fidtable'
stepname = '/CellpyData/step_table'

v_hdr = "Voltage"
c_hdr = "Charge_Capacity"
d_hdr = "Discharge_Capacity"
i_hdr = "Current"


### Defining some functions

In [8]:
def _query_dfdata(store, q):
    return store.select('/CellpyData/dfdata', q)

In [9]:
def _convert_to_specific(s, mass):
    return  s*1000000.0 / mass
    

In [10]:
def _make_selectors(t):
    if not t.lower() in [
        "galvanostatic_charge", "galvanostatic_discharge",
        "cycle", "ocv",
        "discharge", "cv_discharge", "charge", "cv_charge", "ocvrlx_up", "ocvrlx_down",
    ]:
        # error-message here
        return
    selectors = []
    if t.lower() == "galvanostatic_charge":
        selectors.append("charge")
    elif t.lower() == "galvanostatic_discharge": 
        selectors.append("discharge")
    elif t.lower() == "charge": 
        selectors.extend(["charge", "cv_charge"])
    elif t.lower() == "discharge": 
        selectors.extend(["discharge", "cv_discharge"])
    elif t.lower() == "cycle": 
        selectors.extend(["charge", "discharge"])
    elif t.lower() == "ocv": 
        selectors.extend(["ocvrlx_down", "ocvrlx_up"])
    else:
        selectors.append(t.lower())
                
    return(selectors)


In [11]:
def _get_step_table(file_name):
    step_name = '/CellpyData/step_table'
    with pd.HDFStore(file_name) as store:
        step_table = store.select(step_name)
    return step_table


In [12]:
def _filter_step_table(selectors, step_table, cycle_numbers):
    steps = step_table.type.isin(selectors) & step_table.cycle.isin(cycle_numbers)
    return step_table.loc[steps]

In [13]:
def _generate_cycle_data_query(a, headers=None):
    """create cycle query from a step_table DataFrame"""
    a = a[["point_first", "point_last"]]
    if headers is None:
        headers = [
            "Step_Time",
            "Voltage",
            "Charge_Capacity",
            "Discharge_Capacity",
            "Current"
        ]
        
    sections = []
    for points in a.iterrows():
        p1, p2 = points[-1].values
        sections.append(f"(index>={p1} & index<={p2})")
    q = " | ".join(sections)
    
    hdr_txts = []
    for h in headers:
        hdr_txts.append(f"'{h}'")
    hdr_txt = ", ".join(hdr_txts)
    q += f" & columns = [{hdr_txt}]"
    return q

### Testing if the functions work

In [14]:
cell_id = "20180925_sic070_01_cc"
cellpyfile = infodf.cellpy_file_names.loc["20180925_sic070_01_cc"]
mass = infodf.masses.loc[cell_id]
cycle_numbers = [1,2, 3]
step_table = _get_step_table(cellpyfile)

In [15]:
with pd.HDFStore(cellpyfile) as store:
    selectors = _make_selectors("cycle")
    a = _filter_step_table(selectors, step_table, cycle_numbers=cycle_numbers)
    q = _generate_cycle_data_query(a)
    c = _query_dfdata(store, q)
    c["Charge_Capacity"] = _convert_to_specific(c["Charge_Capacity"], mass)
    c["Discharge_Capacity"] = _convert_to_specific(c["Discharge_Capacity"], mass)

In [16]:
c.head()

,Step_Time,Current,Voltage,Charge_Capacity,Discharge_Capacity
Data_Point,,,,,
74,0.0077,-0.000235,3.176647,0.0,0.000082
75,0.0429,-0.000235,3.170548,0.0,0.000466
76,0.0722,-0.000235,3.165090,0.0,0.000876
77,0.0970,-0.000235,3.159763,0.0,0.001181
78,0.1089,-0.000235,3.153211,0.0,0.001317


#### Plotting

In [17]:
c.plot.line(x="Charge_Capacity", y="Voltage", )

FigureCanvasNbAgg()

In [18]:
c.plot.line(x=["Discharge_Capacity", "Charge_Capacity"], y="Voltage", )

FigureCanvasNbAgg()

In [19]:
c.head(10000)

,Step_Time,Current,Voltage,Charge_Capacity,Discharge_Capacity
Data_Point,,,,,
74,0.007700,-0.000235,3.176647,0.000000,0.000082
75,0.042900,-0.000235,3.170548,0.000000,0.000466
76,0.072200,-0.000235,3.165090,0.000000,0.000876
77,0.097000,-0.000235,3.159763,0.000000,0.001181
78,0.108900,-0.000235,3.153211,0.000000,0.001317
79,0.126499,-0.000235,3.146763,0.000000,0.001571
80,0.144200,-0.000235,3.140427,0.000000,0.001801
81,0.162500,-0.000235,3.134152,0.000000,0.002005
82,0.180100,-0.000235,3.127928,0.000000,0.002229
